In [ ]:
# ① 카테고리별 소비 규모 & 빈도 (기본 중의 기본)
# 📊 시각화

# 카테고리별 총 소비금액

# 카테고리별 총 결제 건수

# 왜 중요?
# → 카드 소비 데이터의 ‘시장 규모’를 보여줌
# → 추천의 기준점

In [ ]:
# 시간 단축을 위해 colab에서 아래의 코드로 진행함

import pandas as pd
import glob

DATA_PATH = "/content/drive/MyDrive/data/*.csv"
file_list = sorted(glob.glob(DATA_PATH))

overall_sum = {}

for file in file_list:
    print(f"Processing {file}")

    for chunk in pd.read_csv(
        file,
        usecols=["card_tpbuz_nm_2", "amt", "cnt"],
        chunksize=200_000
    ):
        grouped = chunk.groupby("card_tpbuz_nm_2")[["amt", "cnt"]].sum()

        for cat, row in grouped.iterrows():
            if cat not in overall_sum:
                overall_sum[cat] = row
            else:
                overall_sum[cat] += row
                
df_overall = (
    pd.DataFrame(overall_sum)
    .T
    .sort_values("amt", ascending=False)
)

df_overall

In [ ]:
# 만들어진 df_overall을 csv로 저장하기
import os

OUTPUT_DIR = "/content/drive/MyDrive/data/output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

df_overall_out = df_overall.reset_index()

df_overall_out.columns = ["category", "total_amt", "total_cnt"]

df_overall_out.to_csv(
    f"{OUTPUT_DIR}/overall_category_summary_202201_202511.csv",
    index=False,
    encoding="utf-8-sig"
)

======================= 위쪽이 colab에서 진행한 방법, 아래는 로컬에서 진행하기 위해 읽어오는 방법 및 진행 =======================

In [ ]:
import pandas as pd

OUTPUT_DIR = "../output"

df_overall = pd.read_csv(
    f"{OUTPUT_DIR}/overall_category_summary_202201_202511.csv"
)

df_overall

,category,total_amt,total_cnt
0,한식,14231837129588,444169343
1,고기요리,4832101787774,83635120
2,커피/음료,4450253240273,497049761
3,일식/수산물,3983001301734,79728690
4,닭/오리요리,2583407720693,81527240
5,제과/제빵/떡/케익,2499151364491,190398054
6,분식,2268182376441,159427911
7,간이주점,2158785025691,47313706
8,패스트푸드,1822363675344,152450506
9,별식/퓨전요리,1699396050437,68397969


======================= 속도가 비슷할 거 같아서 로컬로 월별 소비, 결제 횟수 등 확인 =======================

In [4]:
import pandas as pd
import glob
import os

DATA_PATH = "../food/*.csv"
file_list = sorted(glob.glob(DATA_PATH))

file_list[:5], len(file_list)  # 202201 ~ 202511 확인

(['../food\\202201.csv',
  '../food\\202202.csv',
  '../food\\202203.csv',
  '../food\\202204.csv',
  '../food\\202205.csv'],
 47)

In [10]:
monthly_data = []

for file in file_list:
    year_month = os.path.basename(file).replace(".csv", "")

    # 🔑 파일 단위 누적용 Series
    file_sum = None

    for chunk in pd.read_csv(
        file,
        usecols=["card_tpbuz_nm_2", "amt"],
        chunksize=200_000
    ):
        chunk_sum = chunk.groupby("card_tpbuz_nm_2")["amt"].sum()

        if file_sum is None:
            file_sum = chunk_sum
        else:
            # 🔥 핵심: 인덱스 기준 자동 정렬 + 누적
            file_sum = file_sum.add(chunk_sum, fill_value=0)

    # 파일 하나 끝나면 DataFrame으로 변환
    if file_sum is not None:
        file_df = file_sum.reset_index()
        file_df["year_month"] = year_month
        monthly_data.append(file_df)

# ✅ 최종 병합 (단 1번)
df_monthly = pd.concat(monthly_data, ignore_index=True)
df_monthly.columns = ["category", "amt", "year_month"]

In [11]:
df_monthly

,category,amt,year_month
0,간이주점,23371526772,202201
1,고기요리,77821866715,202201
2,닭/오리요리,47026223576,202201
3,별식/퓨전요리,20861113454,202201
4,분식,35337809312,202201
...,...,...,...
559,제과/제빵/떡/케익,54373547439,202511
560,중식,31017255053,202511
561,커피/음료,92025821643,202511
562,패스트푸드,40169578667,202511


In [15]:
OUTPUT_DIR = "../output"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

df_monthly.to_csv(
    f"{OUTPUT_DIR}/monthly_category_amt_202201_202511.csv",
    index=False,
    encoding="utf-8-sig"
)

In [16]:
import pandas as pd

OUTPUT_DIR = "../output"

df_monthly = pd.read_csv(
    f"{OUTPUT_DIR}/monthly_category_amt_202201_202511.csv"
)

df_monthly

,category,amt,year_month
0,간이주점,23371526772,202201
1,고기요리,77821866715,202201
2,닭/오리요리,47026223576,202201
3,별식/퓨전요리,20861113454,202201
4,분식,35337809312,202201
...,...,...,...
559,제과/제빵/떡/케익,54373547439,202511
560,중식,31017255053,202511
561,커피/음료,92025821643,202511
562,패스트푸드,40169578667,202511
